<a href="https://colab.research.google.com/github/devmanfredi/devmanfredi/blob/master/fiis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas yfinance requests beautifulsoup4 plotly openpyxl

In [ ]:
#libs
from datetime import datetime
import pandas as pd
import yfinance as yf
import requests as rq
from bs4 import BeautifulSoup
import plotly
import openpyxl

In [ ]:
#Data Collecting
url = "https://www.fundsexplorer.com.br/ranking"
resp = rq.get(url)
soup = BeautifulSoup(resp.text, "lxml")
table = soup.find_all("table")[0]
dff = pd.read_html(str(table), decimal=',' ,thousands='.')[0]

In [ ]:
#Format columns
columns_formated = ['codigo_fundo','setor','preco_atual','liq_diaria','dividendo','div_yeld','dy_3m_acum','dy_6m_acum','dy_12m_acum','dy_3m_media','dy_6m_media','dy_12m_media','dy_ano','var_preco','rent_periodo','rent_acum','patri_liq','vpa','p_vpa','dypatrimonial','var_patrimonial','rent_patri_periodo','rent_patri_acum','vacancia_fisica','vacancia_financeira','qtd_ativos']
dff.columns = columns_formated
dff

In [ ]:
#to_excel
data_e_hora_atuais = datetime.now()
data_e_hora_em_texto = data_e_hora_atuais.strftime('%d_%m_%Y_%H%M')
name = 'fiis_{}{}'.format(data_e_hora_em_texto, '.xlsx')
print(name)
dff.to_excel(name)

fiis_07_05_2022_1958.xlsx


p1.mudar formatação no excel de . -> , 
p2.abrir documento no excel e mudar para numero
p3.adicionar 4 casas decimais nos numeros que foram formatados
voltar para p1. dessa vez invertido
salvar csv com utf-8 por virgula

In [ ]:
#excel_with_csv
df = pd.read_csv('fiis.csv', decimal=',', thousands='.', delimiter=',')
df

In [ ]:
#get_symbols
symbols = df['codigo_fundo'] + '.SA'
symbols

In [ ]:
def get_funds(name):
    print("name: " , name)
    stock = yf.download(name, start=datetime(2017,12,1), end=datetime.today())
    return stock

stock_dict = {}

for symbol in symbols:
    try:
        stock_dict[symbol] = get_funds(symbol)
    except:
        print("DO NOT FIND FOR TICKET: " , symbol)
print("END")

In [ ]:
returns = pd.DataFrame()

for key in stock_dict.keys():
  returns[key] = stock_dict[key]['Adj Close'].pct_change()

returns.head()

# print(returns)

In [ ]:
discount_dict = {}

def calcula_desconto(df):
  max_price = df['Adj Close'].describe()['75%']
  last_price = df['Adj Close'].iloc()[-1]
  return (last_price - max_price)/max_price

In [ ]:
for tick in stock_dict.keys():
  try:
    discount_dict[tick] = calcula_desconto(stock_dict[tick])
  except:
    discount_dict[tick] = 0

discount_dict

In [ ]:
def get_sector(stock):
  return [x for x in df[symbols == stock]['setor']]

stock_info_df = pd.DataFrame.from_dict(discount_dict, orient='index').reset_index()
stock_info_df.columns = ['ticker', 'discount']
stock_info_df['sector'] = [get_sector(tick)[0] for tick in stock_info_df['ticker']]

stock_info_df

In [ ]:
def get_dy(stock):
  return [x for x in df[symbols == stock]['dividendo']]

stock_info_df['dy'] = [get_dy(tick)[0] for tick in stock_info_df['ticker']]

stock_info_df

In [ ]:
std_list = []

for tick in stock_info_df['ticker']:
  std_list.append(returns[tick].std())

stock_info_df['risk'] = std_list

stock_info_df.sort_values(by='discount').head(10)

In [ ]:
import plotly.express as px
stock_info_df = stock_info_df.dropna()

In [ ]:
fig = px.scatter(stock_info_df, x='risk',y='discount',color='sector', hover_name='ticker', log_x = True)
plotly.offline.plot(fig, filename='opportunities.html')

'opportunities.html'

//analise quantitativa e qaulitativa